## Setup: Import Libraries

In [2]:
import os
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '1'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import asyncio
from dotenv import load_dotenv

from fairlib.utils.document_processor import DocumentProcessor

from fairlib import (
    settings,
    AbstractTool,
    ManagerPlanner,
    HierarchicalAgentRunner,
    Message,
    HuggingFaceAdapter,
    ToolRegistry,
    ToolExecutor,
    WorkingMemory,
    LongTermMemory,
    ChromaDBVectorStore,
    ReActPlanner,
    SimpleAgent,
    SentenceTransformerEmbedder,
    SimpleRetriever,
    KnowledgeBaseQueryTool  # <-- Using the official framework tool
)

# ChromaDB for vector storage
try:
    import chromadb
    CHROMADB_AVAILABLE = True
except ImportError:
    print("Warning: chromadb not installed. Install with: pip install chromadb")
    CHROMADB_AVAILABLE = False

# Load environment variables
load_dotenv()
token = os.getenv("HUGGING_FACE_HUB_TOKEN")

if not token:
    print("Warning: HUGGING_FACE_HUB_TOKEN not found in .env file!")
else:
    print("Token loaded successfully!")

c:\Users\EBliv\Downloads\Computer Science\final-project-aymeric-et-louis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token loaded successfully!


## Loading LLM

In [3]:
print("Loading language model...")
llm = HuggingFaceAdapter(
    model_name="dolphin3-qwen25-3b", 
    auth_token=token,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    max_new_tokens=512,
    repetition_penalty=1.1
)
print("Model loaded!")

Loading language model...
🔧 Loading HuggingFace model: cognitivecomputations/Dolphin3.0-Qwen2.5-3b (quantized=False, stream=False)


Some parameters are on the meta device because they were offloaded to the cpu and disk.
Device set to use cpu


Model loaded!


## Importing Fair LLM

In [4]:
from fairlib import (
    HuggingFaceAdapter,
    ToolRegistry,
    SafeCalculatorTool,
    ToolExecutor,
    WorkingMemory,
    SimpleAgent, 
    SimpleReActPlanner,
    RoleDefinition
)

print("All components imported!")

All components imported!


## Tool Creation:

In [16]:
import json
import os
from pathlib import Path
from typing import Dict

# ---- Optional: keep your safe soundfile-based saver (avoids torchaudio I/O issues) ----
import numpy as np
import soundfile as sf

def _safe_save_audio(wav, path, samplerate, **kwargs):
    wav_np = wav.detach().cpu().numpy()
    if wav_np.ndim == 1:
        wav_np = wav_np[np.newaxis, :]
    sf.write(path, wav_np.T, samplerate)
    print(f"✅ Saved safely: {path}")

# ---------------------------------------------------------------------------------------

class AbstractTool:
    """Minimal stand-in so this example is self-contained."""
    name: str
    description: str
    def use(self, tool_input: str) -> str:
        raise NotImplementedError

class SeparateStemsTool(AbstractTool):
    """Tool for separating music into stems using Demucs"""

    name = "separate_stems"
    description = (
        "Separates an audio file into stems (vocals, drums, bass, etc.) using Demucs.\n"
        "Input (JSON string): { 'input_file': 'path/to/audio.mp3', "
        "'output_dir': 'separated_stems', 'model_name': 'htdemucs_6s' }.\n"
        "Returns: a formatted report with saved file paths."
    )

    def _separate_stems(self, input_file: str, output_dir: str = "separated_stems",
                        model_name: str = "htdemucs") -> Dict[str, str]:
        """
        Core separation routine. Loads Demucs, runs separation, and saves stems.
        Returns a dict of {stem_name: path}.
        """
        # Lazy imports so importing the tool is cheap
        import torch
        import librosa
        import numpy as np
        from demucs.pretrained import get_model
        from demucs.apply import apply_model
        from demucs import audio as demucs_audio
        import torchaudio

        # Monkey-patch Demucs' save_audio to our safe soundfile-based version
        demucs_audio.save_audio = _safe_save_audio

        if not os.path.exists(input_file):
            raise FileNotFoundError(f"Input file not found: {input_file}")

        # Load audio (stereo @ 44.1k)
        y, sr = librosa.load(input_file, sr=44100, mono=False)
        if y.ndim == 1:
            y = y[np.newaxis, :]  # (channels, samples)
        wav = torch.from_numpy(y).float().unsqueeze(0)  # (1, C, T)

        # Load model
        model = get_model(model_name)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model.to(device)

        # Resample if needed
        if sr != model.samplerate:
            resampler = torchaudio.transforms.Resample(sr, model.samplerate)
            wav = resampler(wav)
            sr = model.samplerate

        wav = wav.to(device)

        # Separate
        with torch.no_grad():
            sources = apply_model(model, wav, device=device)  # (1, nsrc, C, T)

        # Save stems
        out_dir = Path(output_dir)
        out_dir.mkdir(parents=True, exist_ok=True)
        base = Path(input_file).stem
        sources = sources.cpu()

        stem_paths: Dict[str, str] = {}
        print(f"\nSaving stems to: {out_dir}")
        for i, stem_name in enumerate(model.sources):
            out_path = out_dir / f"{base}_{stem_name}.wav"
            # sources[0, i] -> (C, T)
            _safe_save_audio(sources[0, i], str(out_path), sr)
            stem_paths[stem_name] = str(out_path)

        return stem_paths

    def use(self, tool_input: str) -> str:
        """
        Run the tool.
        Expects a JSON string like:
          {"input_file":"dreams.mp3","output_dir":"separated_stems","model_name":"htdemucs_6s"}
        Returns a human-readable report.
        """
        # Parse inputs
        try:
            params = json.loads(tool_input) if tool_input.strip() else {}
            input_file = params.get("input_file")
            output_dir = params.get("output_dir", "separated_stems")
            model_name = params.get("model_name", "htdemucs")
        except json.JSONDecodeError:
            return (
                "Input parsing error: expected a JSON string, e.g.\n"
                '{"input_file":"dreams.mp3","output_dir":"separated_stems","model_name":"htdemucs_6s"}'
            )

        # Validate required arg
        if not input_file:
            return "Missing required field: 'input_file'"

        # Dependency sanity check
        missing = []
        for pkg in ["demucs", "librosa", "soundfile", "torch", "torchaudio"]:
            try:
                __import__(pkg)
            except Exception:
                missing.append(pkg)
        if missing:
            return (
                "Missing required packages: " + ", ".join(missing) +
                "\nInstall with: pip install demucs librosa soundfile torch torchaudio"
            )

        # Run separation
        try:
            stem_paths = self._separate_stems(
                input_file=input_file,
                output_dir=output_dir,
                model_name=model_name,
            )
        except FileNotFoundError as e:
            return f"Error: {e}"
        except Exception as e:
            return f"Separation failed: {type(e).__name__}: {e}"

        # Build report
        lines = [
            "Stem Separation Complete ✅",
            f"- Input: {input_file}",
            f"- Model: {model_name}",
            f"- Output directory: {output_dir}",
            "",
            "Saved stems:"
        ]
        for name, path in stem_paths.items():
            lines.append(f"  • {name:10s} -> {path}")
        return "\n".join(lines)

SeparateStemsTool = SeparateStemsTool()

print("✓ Specialized tools created!")

✓ Specialized tools created!


## Giving agent tools

In [ ]:
# Create a registry to hold all tools
tool_registry = ToolRegistry()

# Instantiate the tool (lowercase variable so you don't shadow the class)
separate_stems_tool = SeparateStemsTool()

# Register it so the agent knows it exists
tool_registry.register_tool(separate_stems_tool)

# See what tools are available
available_tools = [tool.name for tool in tool_registry.get_all_tools().values()]
print(f"Agent's tools: {available_tools}")

SeparateStemsTool -> <__main__.SeparateStemsTool object at 0x00000237074532C0> | type: <class '__main__.SeparateStemsTool'>
isclass? False
Deleted shadowed SeparateStemsTool binding.


AttributeError: 'SeparateStemsTool' object has no attribute 'name'

ModuleNotFoundError: No module named 'fairlib.modules.action.tools.abstract'